In [1]:
!git clone -b atm-10 http://github.com/Zapy67/ATML-PA-2

Cloning into 'ATML-PA-2'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17 (delta 2), reused 10 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 211.62 KiB | 5.88 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [19]:
!git pull http://github.com/Zapy67/ATML-PA-2 atm-10

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 2), reused 6 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 2.09 KiB | 1.04 MiB/s, done.
From http://github.com/Zapy67/ATML-PA-2
 * branch            atm-10     -> FETCH_HEAD
Updating 016d1a9..bb43b04
Fast-forward
 DAN/dan.ipynb        | 122 +++++++++++++++++++++++++++++++++++++++++++--------
 DAN/dataset.py       |   3 ++
 DAN/requirements.txt |   2 -
 3 files changed, 106 insertions(+), 21 deletions(-)


In [22]:
!pip install deeplake

  Using cached deeplake-4.3.4-cp311-cp311-manylinux2014_x86_64.whl.metadata (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 34.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 67.8 MB/s eta 0:00:00:00:01:01m


In [ ]:
import os

os.chdir('./ATML-PA-2/DAN')

FileNotFoundError: [Errno 2] No such file or directory: 'ATML-PA-2/DAN'

In [23]:
ls

architecture.py  dataset.py    requirements.txt  utils.py
dan.ipynb        __pycache__/  train.py


In [24]:
import torch
import architecture
from architecture import dan_resnet



In [25]:
from dataset import source_loader, target_loader, ds






Exception: The API for Deep Lake 4.0 has changed significantly, including the `load` method being replaced by `open`. To continue using Deep Lake 3.x, use `pip install "deeplake<4"`. For information on migrating your code, see https://docs.deeplake.ai/latest/details/v3_conversion/